In [4]:
import requests
import pandas as pd

# Riot API 키 설정
api_key = 'RGAPI-df276c30-6a3a-445c-aa0e-e28dcd04e63d'
headers = {
    'X-Riot-Token': api_key
}

# 챔피언 매치 정보 가져오기 (예시: 챔피언 1번의 최근 매치)
summoner_name = '키보드에음료쏟아서고장내는이도윤'  # 소환사 이름
region = 'ASIA'  # 지역 코드
url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'

# 소환사 ID 얻기
response = requests.get(url, headers=headers)
summoner_data = response.json()
summoner_id = summoner_data['id']

# 매치 리스트 가져오기
matches_url = f'https://{region}.api.riotgames.com/lol/match/v4/matchlists/by-account/{summoner_id}'
matches_response = requests.get(matches_url, headers=headers)
matches_data = matches_response.json()

# 데이터프레임으로 변환
matches_df = pd.DataFrame(matches_data['matches'])

{'status': {'message': 'Forbidden', 'status_code': 403}}


KeyError: 'id'

In [ ]:
# 데이터프레임을 CSV 파일로 저장
csv_filename = 'matches.csv'
matches_df.to_csv(csv_filename, index=False)
print(f'Data saved to {csv_filename}')

In [ ]:
from kafka import KafkaProducer
import json

# Kafka 프로듀서 설정
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# CSV 파일을 읽어와서 Kafka로 전송
csv_data = pd.read_csv(csv_filename)

for index, row in csv_data.iterrows():
    # 필요한 정보를 JSON 형식으로 변환하여 전송
    message = row.to_dict()
    producer.send('riot-matches', message)

producer.flush()
producer.close()
print("Data sent to Kafka")

In [ ]:

from kafka import KafkaConsumer
from hdfs import InsecureClient

# Kafka 컨슈머 설정
consumer = KafkaConsumer(
    'riot-matches',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    group_id='riot-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

# HDFS 클라이언트 설정
hdfs_client = InsecureClient('http://localhost:50070', user='your_hadoop_user')

# 데이터 HDFS에 저장
hdfs_file_path = '/user/your_hadoop_user/riot/matches.csv'

with hdfs_client.write(hdfs_file_path, encoding='utf-8') as writer:
    for message in consumer:
        writer.write(json.dumps(message.value) + '\n')

print(f"Data written to HDFS at {hdfs_file_path}")